In [1]:
import joblib
import pandas as pd
import unicodedata
import re
import pickle
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
pipeline_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/notebooks/fabian/randomforestmurspipeline.pkl"

if os.path.exists(pipeline_path):
	pipeline = joblib.load(pipeline_path)
else:
	print(f"File not found: {pipeline_path}")

In [3]:
csv_path = "/Users/fabiancordenod/code/fqbq69/BIMpredict-/raw_data/datatest/testpredict1/murs1.csv"
df_test = pd.read_csv(csv_path, delimiter=';')

In [4]:
df_test = pd.read_csv(csv_path, delimiter=';')

In [5]:
def clean_col(col):
    col = ''.join(c for c in unicodedata.normalize('NFD', col) if unicodedata.category(c) != 'Mn')
    col = col.lower()
    col = re.sub(r"[ \-\(\)]", "_", col)
    col = re.sub(r"[^a-z0-9_]", "", col)
    col = re.sub(r"_+", "_", col)
    col = col.strip("_")
    return col

df_test.columns = [clean_col(c) for c in df_test.columns]

In [6]:
# Les targets à prédire (après nettoyage)
targets = [
    "012ec_ouvrage",
    "013ec_localisation",
    "014ec_mode_constructif"
]

# Colonnes explicatives attendues par le pipeline
preprocessor = pipeline.named_steps['preprocessor']
features = []
for name, trans, cols in preprocessor.transformers_:
    if cols is not None:
        features.extend(cols)

# S'assurer que toutes les colonnes sont présentes
for col in features:
    if col not in df_test.columns:
        df_test[col] = np.nan
X_test = df_test[features].copy()


/var/folders/x6/j7yc7yjj3xd1bjpj4469g9880000gn/T/ipykernel_79158/3085175220.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[col] = np.nan
/var/folders/x6/j7yc7yjj3xd1bjpj4469g9880000gn/T/ipykernel_79158/3085175220.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_test[col] = np.nan
/var/folders/x6/j7yc7yjj3xd1bjpj4469g9880000gn/T/ipykernel_79158/3085175220.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Co

In [7]:
X_test

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,id,sols_en_intersection,sols_coupes_u,sols_coupants_u,fenetres,portes,ouvertures,murs_imbriques,section,type_predefini_difc,...,ter_classe,ter_niveau,ter_intext,ter_apparent,prefixe,12,13,18,19,20
Id,011EC_Lot,012EC_Ouvrage,013EC_Localisation,014EC_Mode Constructif,Nom,Hauteur,Epaisseur,AI,AS,Sols en intersection,Sols coupés (u),Sols coupés (Ids),Sols coupants (u),Sols coupants (Ids),Sol au-dessus,Sol en-dessous,Fenêtres,Portes,Ouvertures,Murs imbriqués,Mur multicouche,Mur empilé,Profil modifié,Image,Catégorie,Section,Type prédéfini d'IFC,Exporter au format IFC sous,Exporter au format IFC,IfcGUID,Variantes,Extension inférieure,Extension supérieure,Volume,Surface,Phase de démolition,Phase de création,Commentaires,Longueur,Famille et type,Famille,Type,Nom de la famille,Nom du type,ID du type,Lié au volume,Structure,Identifiant,Ligne de justification,Utilisation structurelle,Partie inférieure attachée,Partie supérieure attachée,Décalage supérieur,Décalage inférieur,Contrainte inférieure,Hauteur non contrainte,Contrainte supérieure,Limite de pièce,Visible dans les nomenclatures,Légende détails,O - Finition,O - Teinte,Composition,IdGtcParameter,IdGtcParameterCreationTime,EIF_STR - Impact,Image du type,Note d'identification,Type: Type prédéfini d'IFC,Exporter le type au format IFC sous,Exporter le type au format IFC,Type IfcGUID,Modèle,Fabricant,Commentaires du type,URL,Description,Matériau structurel,Rugosité,Coefficient d'absorbance,Masse thermique,Résistance thermique (R),Coefficient de transfert thermique (U),Description de l'assemblage,Code d'assemblage,Retournement aux insertions,Retournement aux extrémités,Couleur vue détail faible,Motif vue détail faible,Marque de type,Protection contre l'incendie,Coût,Fonction,Largeur,T - Teinte,T - Finition,T - Accessoire,Finition,Teinte,Reférence,A une association,Enrobage d'armature - Autres faces,Enrobage d'armature - Face intérieure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
636502,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 62,"3,29000000000008","0,62","-2,94000000000002","0,350000000000055",0,0,NaN,1,636929,False,True,0,0,0,0,False,False,False,<Aucun>,Murs,1,NaN,NaN,0,0e30kvNPT6bRA1RB1hfNPj,-1,0,0,"65,6407640000018","105,872200000003",Aucun(e),Nouvelle construction,NaN,"31,5600000000001",Mur de base: EIF_STR - Mur ep. 62,Mur de base,EIF_STR - Mur ep. 62,NaN,NaN,928776,0,0,NaN,2,0,0,0,"0,0500000000002962","0,31",A&B_SS1,"3,29000000000029",Jusqu'au niveau: A&B_RDC,1,0,NaN,NaN,NaN,NaN,0,0,1,<Aucun>,NaN,NaN,NaN,0,03kNMoz5n75Opf4aeF3oyx,NaN,NaN,Voile ép =62,NaN,NaN,<Par catégorie>,1,"0,1",936882,"0,592734225621415","1,68709677419355",NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,"0,62",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
636503,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 62,"3,28867628056769","0,62","-2,94000000000002","0,348676280567673",1,0,NaN,2,"646972,647247",True,True,0,0,0,0,False,False,False,<Aucun>,Murs,1,NaN,NaN,0,0e30kvNPT6bRA1RB1hfNPi,-1,0,0,"37,8929894170593","61,4693363416546",Aucun(e),Nouvelle construction,NaN,"19,0900000000005",Mur de base: EIF_STR - Mur ep. 62,Mur de base,EIF_STR - Mur ep. 62,NaN,NaN,928776,0,0,NaN,2,0,0,0,"0,0486762805679142","0,31",A&B_SS1,"3,2886762805679",Jusqu'au niveau: A&B_RDC,1,0,NaN,NaN,NaN,NaN,0,0,1,<Aucun>,NaN,NaN,NaN,0,03kNMoz5n75Opf4aeF3oyx,NaN,NaN,Voile ép =62,NaN,NaN,<Par catégorie>,1,"0,1",936882,"0,592734225621415","1,68709677419355",NaN,NaN,0,0,0,NaN,NaN,NaN,0,1,"0,62",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
636504,NaN,NaN,NaN,NaN,EIF_STR - Mur ep. 62,"3,11000000000025","0,62","-2,94000000000002","0,170000000000235",0,0,NaN,1,637095,False,True,0,0,0,0,False,False,False,<Aucun>,Murs,1,NaN,NaN,0,0e30kvNPT6bRA1RB1hfNPZ,-1,0,0,"48,6292040020899","78,4342000033706",Aucun(e),

In [8]:
y_pred = pipeline.predict(X_test)

/Users/fabiancordenod/.pyenv/versions/3.10.6/envs/BIMpredict2/lib/python3.10/site-packages/sklearn/impute/_base.py:558: UserWarning: Skipping features without any observed values: ['type_predefini_difc' 'exporter_au_format_ifc_sous' 'nom_de_la_famille'
 'nom_du_type' '021ec_specificite' '041ec_option_logetex_1'
 '042ec_option_logetex_2' '043ec_option_logetex_3'
 '051ec_date_realisation' '054ec_jour_coulage' '055ec_jour_coulage_cumule'
 '084ec_clef_planification' '085ec_clef_grue' '024ec_finition_go'
 '056ec_heure_grue' '087ec_clef_ab' '088ec_clef_bmo'
 'note_didentification' 'type_type_predefini_difc' 'fabricant' 'url'
 'marque_de_type' 'composition' 'ifcexportas' 'id_montage'
 'designation_systeme' 'ref_dt' 'modifie_par' 'zone' 'pj_degre_coupe_feu'
 'affaiblissement_acoustique' 'pj_identification_local' 'rdd_code_topo'
 'acoustique' 'installation_fabrication' 'construction_details'
 'installation_phase' 'ecno_type_ferraillage' 'ifcexporttype' 'zonename'
 'str_materiau' 'str_ep' 'str_h

In [9]:
# Mettre les prédictions dans un DataFrame
df_pred = pd.DataFrame(y_pred[:, :len(targets)], columns=targets)

# Afficher les premières lignes
print(df_pred.head())

# Sauvegarder si besoin
df_pred.to_csv("resultats_predictions_murs1.csv", index=False)

  012ec_ouvrage 013ec_localisation 014ec_mode_constructif
0           MUR          INTERIEUR                 BANCHE
1           MUR          INTERIEUR                 BANCHE
2           MUR          INTERIEUR                 BANCHE
3           MUR          INTERIEUR                 BANCHE
4           MUR          INTERIEUR                 BANCHE


In [10]:
df_pred.head()

,012ec_ouvrage,013ec_localisation,014ec_mode_constructif
0,MUR,INTERIEUR,BANCHE
1,MUR,INTERIEUR,BANCHE
2,MUR,INTERIEUR,BANCHE
3,MUR,INTERIEUR,BANCHE
4,MUR,INTERIEUR,BANCHE
